# 🔰PyTorchでニューラルネットワーク基礎 #14 【音楽分類02・conv1d】

## 内容
* Qiitaの記事と連動しています
* 1次元畳み込みを利用した音楽ジャンル分類

## 注意
* [Kaggle: GTZAN Dataset - Music Genre Classification](https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification)からGTZANデータセットをダウンロードする必要があります。
* データセットtrain_data_10.npzを作成しないと動作しないので注意してください

### データについて
* train_data_10.npz : 各90ファイル、10秒、10分類
* sampling rate 22050
* 系列長の時間：10秒 (22050*10)

### テスト精度
* 訓練精度：99%
* テスト精度：70〜75%　（結構大きく変わります。色々工夫する余地あり）
* 学習ループ100回ですが、もう少しあったほうがいいかも😓

### その他
* GPU利用時で約20分
* Google Colabでも動作します。
* モデルの構造を表示させるtorchinfoライブラリを使う場合は事前にインストールが必要です。
> pip install torchinfo



In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset  # ミニバッチの利用
from sklearn.model_selection import train_test_split

In [2]:
data = np.load("./data/train_data_10.npz")
x = data["x"]
t = data["t"]
x.shape, t.shape

((2691, 220500), (2691,))

### データセットの準備
* データをtorch.Tensorに変更
* train_test_splitで学習用とテスト用に分割
* DataLoaderでミニバッチ学習の準備

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

X = torch.FloatTensor(x).view(x.shape[0], 1, x.shape[1])
T = torch.LongTensor(t)

x_train, x_test, t_train, t_test = train_test_split(X, T, test_size=0.2, stratify=t, random_state=55)

# ミニバッチに区分けする
mini_batch_size = 100
train_data = TensorDataset(x_train, t_train)
train_loader = DataLoader(train_data, batch_size=mini_batch_size, shuffle=True, drop_last=True)

device: cuda


## ネットワーク構造
* nn.Sequentialを利用して特徴量抽出ブロックと分類ブロックに分けて記述

In [5]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()        
        self.features = nn.Sequential(
            # 第1ブロック
            nn.Conv1d(1, 64, kernel_size=1000, stride=16),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=2),
            
            # 第2ブロック
            nn.Conv1d(64, 128, kernel_size=500, stride=8),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=2),
            
            # 第3ブロック
            nn.Conv1d(128, 256, kernel_size=16, stride=4),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(8)  # 固定サイズの出力
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 8, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [6]:
model = DNN()
model.to(device)

DNN(
  (features): Sequential(
    (0): Conv1d(1, 64, kernel_size=(1000,), stride=(16,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(64, 128, kernel_size=(500,), stride=(8,))
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(128, 256, kernel_size=(16,), stride=(4,))
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): AdaptiveAvgPool1d(output_size=8)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=512, bias=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Dropout(p=0.3,

### ネットワーク構造の表示
* torchinfoライブラリを利用する

In [7]:
from torchinfo import summary
summary(model, (1,1,220500))  # (バッチサイズ, チャンネル数, 系列長)

Layer (type:depth-idx)                   Output Shape              Param #
DNN                                      [1, 10]                   --
├─Sequential: 1-1                        [1, 256, 8]               --
│    └─Conv1d: 2-1                       [1, 64, 13719]            64,064
│    └─BatchNorm1d: 2-2                  [1, 64, 13719]            128
│    └─ReLU: 2-3                         [1, 64, 13719]            --
│    └─MaxPool1d: 2-4                    [1, 64, 6858]             --
│    └─Conv1d: 2-5                       [1, 128, 795]             4,096,128
│    └─BatchNorm1d: 2-6                  [1, 128, 795]             256
│    └─ReLU: 2-7                         [1, 128, 795]             --
│    └─MaxPool1d: 2-8                    [1, 128, 396]             --
│    └─Conv1d: 2-9                       [1, 256, 96]              524,544
│    └─BatchNorm1d: 2-10                 [1, 256, 96]              512
│    └─ReLU: 2-11                        [1, 256, 96]             

In [9]:
# 損失関数と最適化関数の定義
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

In [10]:
def accuracy(y, t):
    _,argmax_list = torch.max(y, dim=1)
    accuracy = sum(argmax_list == t).item()/len(t)
    return accuracy


In [11]:
LOOP = 100
model.train()
for epoch in range(LOOP):
    # ミニバッチの処理
    total_loss = 0  # 損失の累計を計算
    total_acc = 0   # 精度の累計を計算
    cnt = 0   # ミニバッチでの繰り返し回数 cntで割れば平均になる
    
    for x, t in train_loader:
        x = x.to(device)
        t = t.to(device)
        y = model(x)
        loss = criterion(y, t)
        acc  = accuracy(y, t)

        total_loss += loss.item()
        total_acc += acc
        cnt += 1

        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()

    if (epoch+1)%20 == 0:
        print(f"{epoch}: loss: {total_loss/cnt},\tacc:{total_acc/cnt}")   # 損失と精度の表示

19: loss: 0.4493510623772939,	acc:0.927142857142857
39: loss: 0.1916269390355973,	acc:0.9638095238095238
59: loss: 0.08933362843734878,	acc:0.9880952380952381
79: loss: 0.06818140094124135,	acc:0.9904761904761905
99: loss: 0.032543914614333994,	acc:0.9952380952380951


# テスト用データを使って検証

In [19]:
model.eval()
with torch.inference_mode():
    x_test = x_test.to(device)
    t_test = t_test.to(device)
    y_test = model(x_test)
test_acc = accuracy(y_test, t_test)
test_acc

0.7495361781076066